# OSTK Data Cleaning Automation Tool

## Overview

This notebook automates the process of cleaning and transforming data for OSTK product listings. The objective is to ensure the data meets required quality standards before analysis and reporting.

Key tasks include:
- Loading and normalizing raw JSON data
- Converting nested price data into flat tables
- Parsing specifications into dictionaries and then DataFrames
- Merging parsed data with original DataFrame
- Renaming and dropping duplicate or unwanted columns
- Exporting the final dataset to Excel and applying styling



Following has applied to improve efficiency:
- Remove unused imports
- Clean dictionary-maker logic 
- Consolidate the creation of spec_data into single loop
- Rename variable for clarity
- Optimize Excel styling

## 1. Import Libraries and Load Data

In [25]:
# Import necessary libraries
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font
from openpyxl.styles import numbers

In [26]:
# Load the JSON file
df = pd.read_json("../data/raw/snap_m82yajni2aaxivr8hc.json")

In [27]:
list(df.columns)

['availability',
 'average_rating',
 'breadcrumbs',
 'category',
 'combinations',
 'description',
 'image_count',
 'is_video',
 'name',
 'number_of_reviews',
 'promo_price',
 'regular_price',
 'reviews',
 'sale_tag',
 'specifications',
 'url']

## 2. Data Cleaning and Preprocessing

In [28]:
# Normalize nested JSON fields
regular_price = pd.json_normalize(df['regular_price'])
promo_price = pd.json_normalize(df['promo_price'])
link = pd.json_normalize(df['url'])

# Concateante the original data with normalized columns
df_new = pd.concat([df,regular_price,promo_price,link],axis=1)

In [29]:
list(df_new.columns)

['availability',
 'average_rating',
 'breadcrumbs',
 'category',
 'combinations',
 'description',
 'image_count',
 'is_video',
 'name',
 'number_of_reviews',
 'promo_price',
 'regular_price',
 'reviews',
 'sale_tag',
 'specifications',
 'url',
 'currency',
 'symbol',
 'value',
 'currency',
 'symbol',
 'value']

## 3.  Convert Specifications to a DataFrame

In [30]:
# Function to convert a list of strings (each in key:value format) into a dictionary
def dictionary_maker(record):
    if not record or not isinstance(record, list):
        return {}
    
    keys, values = [], []
    for item in record:
        try:
            key, val = item.split(":", 1)
            keys.append(key.strip())
            values.append(val.strip())
        except:
            continue
    return dict(zip(keys, values))

In [31]:
# Convert 'specifications' into dictionaries
df_new['specifications'] = df_new['specifications'].apply(dictionary_maker)

# Clean empty keys in 'specifications'
for idx in range(len(df_new)):
    try:
        specs_dict = df_new.at[idx, 'specifications']
        if '' in specs_dict:
            specs_dict['Item #'] = specs_dict['']
            del specs_dict['']
    except:
        pass

In [32]:
# Convert each row's 'specifications' into a DataFrame indexed by 'url'
spec_df_list = []

for i in df_new.index:
    try:
        row_specs = df_new.at[i, 'specifications']
        row_url = df_new.at[i, 'url']
        temp_df = pd.DataFrame([row_specs], index=[row_url])
        spec_df_list.append(temp_df)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        continue

specs_merged = pd.concat(spec_df_list).reset_index().rename(columns={'index': 'url'})

# Merge specs DataFrame back into main DataFrame
df_final = pd.merge(df_new, specs_merged, on='url', how='left')

# Replace spaces with underscores in column names
df_final.columns = [col.replace(' ', '_') if isinstance(col, str) else col for col in df_final.columns]

# Retrieve the list of columns from the copied DataFrame
df_final_columns = df_final.columns.to_list()

## 4. Handle Duplicate Columns and Rename Specific Ones

In [35]:
# Identify duplicate columns
dupe_cols = df_final.columns[df_final.columns.duplicated()].unique()
if len(dupe_cols) > 0:
    print(f"Duplicate columns: {dupe_cols}")

# Append index to duplicates
df_final.columns = [
    f"{col}_{i}" if col in dupe_cols else col
    for i, col in enumerate(df_final.columns)
]

In [36]:
print(list(df_final.columns))

['availability', 'average_rating', 'breadcrumbs', 'category', 'combinations', 'description', 'image_count', 'is_video', 'name', 'number_of_reviews', 'promo_price', 'regular_price', 'reviews', 'sale_tag', 'specifications', 'url', 'currency_16', 'symbol_17', 'value_18', 'currency_19', 'symbol_20', 'value_21', 'Item_#', 'Dimensions', 'Style', 'Material', 'Finish', 'Set_Size', 'Frame_Material', 'Assembly', 'Chair_Back_Height', 'Arms', 'Color', 'Number_of_Pieces', 'Product_Features', 'Seat_Height', 'Seat_Material', 'Seat_Style', 'Stool_Base', 'Weight_Capacity', 'Warranty', 'Model_Number', 'Bed_Design', 'Size', 'Slats_Included', 'Wood_Species', 'Back_Style', 'Upholstery_Material', 'Base_Type', 'Chair_Material', 'Seats_Up_To', 'Set_Design', 'Shape', 'Table_Frame_Material', 'Table_Height', 'Chair_Type', 'Cushion_Material', 'Design', 'Top_Material', 'Type', 'Set_Includes', 'Pattern', 'Length', 'Chair_Back_Style', 'Expandable', 'Table_Top_Material', 'Storage_Type', 'Print', 'Arm_Style', 'Cushion

In [37]:
# Rename the specific columns
df_final.rename(columns={'value_18': 'regular_retail_price', 
                         'value_21': 'discounted_retail_price'}, 
                         inplace=True)

# List the current columns for review
print(list(df_final.columns))

['availability', 'average_rating', 'breadcrumbs', 'category', 'combinations', 'description', 'image_count', 'is_video', 'name', 'number_of_reviews', 'promo_price', 'regular_price', 'reviews', 'sale_tag', 'specifications', 'url', 'currency_16', 'symbol_17', 'regular_retail_price', 'currency_19', 'symbol_20', 'discounted_retail_price', 'Item_#', 'Dimensions', 'Style', 'Material', 'Finish', 'Set_Size', 'Frame_Material', 'Assembly', 'Chair_Back_Height', 'Arms', 'Color', 'Number_of_Pieces', 'Product_Features', 'Seat_Height', 'Seat_Material', 'Seat_Style', 'Stool_Base', 'Weight_Capacity', 'Warranty', 'Model_Number', 'Bed_Design', 'Size', 'Slats_Included', 'Wood_Species', 'Back_Style', 'Upholstery_Material', 'Base_Type', 'Chair_Material', 'Seats_Up_To', 'Set_Design', 'Shape', 'Table_Frame_Material', 'Table_Height', 'Chair_Type', 'Cushion_Material', 'Design', 'Top_Material', 'Type', 'Set_Includes', 'Pattern', 'Length', 'Chair_Back_Style', 'Expandable', 'Table_Top_Material', 'Storage_Type', 'Pr

In [38]:
# Drop unwanted columns
cols_to_drop=[
    'currency_16','symbol_17','currency_19',
    'symbol_20','promo_price','regular_price'
    ]
df_final.drop(columns=cols_to_drop, errors='ignore', inplace=True)

# Reorder columns
main_cols = [
    'Item_#', 'Model_Number', 'name', 'category', 'combinations', 
    'description','regular_retail_price','discounted_retail_price'
    ]
other_cols = [c for c in df_final.columns if c not in main_cols]
df_final = df_final[main_cols + other_cols]

## 5. Finalize DataFrame and Export to Excel

In [39]:
# Export the processed DataFrame to an Excel file
df_final.to_excel('../data/processed/overstock_retail_data_3132025.xlsx')

## 6. Load Excel Workbook and Apply Styling

In [40]:
# Load the exported Excel workbook
wb =load_workbook(filename = '../data/processed/overstock_retail_data_3132025.xlsx')

# Select the active worksheet
ws = wb.active

# Apply an auto-filter
ws.auto_filter.ref = ws.dimensions

In [41]:
# Define a header font style
font = Font(size=15, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='FF000000')

# Define text wrapping alignment
wrap = Alignment(wrapText=True,horizontal='left')

# Define left alignment for cells
left_alignment = Alignment(horizontal='left')

# Define a fill pattern for header cells
fill = PatternFill("solid", fgColor="00CCFFCC")

# Define thin borders for cells
top=Side(border_style='thin',color="FF000000")
bottom=Side(border_style='thin', color="FF000000")
left = Side(border_style='thin', color="FF000000")
right = Side(border_style='thin', color="FF000000")
border=Border(top=top,bottom=bottom,left=left,right=right)

In [42]:
# Get the total number of rows in the worksheet
last_row = ws.max_row

# Set a standard row height for all rows
for i in range(2,last_row+1):
    ws.row_dimensions[i].height = 15

# Apply number formatting to column B
for cell in ws["B:B"]:
    cell.number_format = numbers.FORMAT_NUMBER

# Apply left alignment and thin borders to all cells 
for rows in ws.iter_rows(min_row=1, max_row=last_row):
    for cell in rows:
         cell.alignment = left_alignment
         cell.border = border

# Format the header row
for cell in ws["1:1"]:
    cell.font = font
    cell.fill = fill

## 7. Apply Additional Alignment for Specific Columns

In [43]:
# Enable text wrapping for specific columns
for col in ['D', 'E', 'I', 'O', 'P']:
    for cell in ws[col]:
        cell.alignment = wrap

## 8. Freeze Panes and Set Column Widths

In [44]:
# Freeze panes to keep the header visible when scrolling
ws.freeze_panes = ws["B2"]

# Define column widths
col_widths = {
    "B": 20, "C": 50, "D": 30, "E": 60, "F": 60,
    "G": 20, "H": 20, "I": 20, "J": 20, "K": 20,
    "L": 20, "M": 20, "N": 20, "O": 30, "P": 20,
    "Q": 100, "R": 40, "S":40
}

# Apply column widths
for col, width in col_widths.items():
    ws.column_dimensions[col].width = width

## 9. Save the Styled Workbook

In [45]:
wb.save("../data/processed/overstock_retail_data_3132025_styled.xlsx")